In [ ]:
# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Running in Google Colab")

    # Mount Google Drive for persistent storage
    from google.colab import drive
    drive.mount('/content/drive')

    # Create project directory in Google Drive
    import os
    PROJECT_DIR = '/content/drive/My Drive/Resume_Analyzer'
    os.makedirs(PROJECT_DIR, exist_ok=True)
    os.chdir(PROJECT_DIR)
    print(f"📁 Project directory: {PROJECT_DIR}")

except ImportError:
    IN_COLAB = False
    PROJECT_DIR = '.'
    print("⚠️ Not running in Colab - using local directory")


In [ ]:
!pip install --upgrade pip
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q numpy pandas scikit-learn scipy tqdm matplotlib seaborn spacy fastapi uvicorn python-multipart nest-asyncio sentence-transformers transformers
!pip install pdfplumber python-docx textstat



In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import pandas as pd
import numpy as np
import re
import io
import logging
import os
import json
import pickle
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union
from dataclasses import dataclass
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# NLP and ML
from sentence_transformers import SentenceTransformer
import spacy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr

# Document processing
import pdfplumber
from docx import Document
import textstat

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm  # Auto-detect Colab

# Colab specific imports
if IN_COLAB:
    from google.colab import files
    import nest_asyncio
    nest_asyncio.apply()  # Allow nested event loops in Colab

# Kaggle dataset
import kagglehub

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger("ResumeAnalyzer")

@dataclass
class ColabConfig:
    """Colab-optimized configuration"""

    # Model architecture (smaller for Colab memory limits)
    SENTENCE_TRANSFORMER = "sentence-transformers/all-MiniLM-L6-v2"  # Smaller model
    EMBEDDING_DIM = 384  # Reduced for memory
    HIDDEN_DIMS = [256, 128, 64]  # Smaller network
    DROPOUT_RATES = [0.3, 0.2, 0.1]

    # Training parameters (Colab-friendly)
    BATCH_SIZE = 8  # Smaller batch size for Colab
    LEARNING_RATE = 2e-5
    WEIGHT_DECAY = 0.01
    EPOCHS = 20  # Fewer epochs for faster training
    GRADIENT_CLIP = 1.0

    # Data augmentation
    AUGMENTATION_FACTOR = 2  # Less augmentation for speed
    NOISE_FACTOR = 0.05

    # Feature weights
    FEATURE_WEIGHTS = {
        'skills': 0.25, 'experience': 0.20, 'education': 0.15,
        'achievements': 0.15, 'formatting': 0.10, 'keywords': 0.10, 'readability': 0.05
    }

    # Device and paths
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    NUM_WORKERS = 2  # Colab-friendly

    # Paths in Google Drive
    MODEL_DIR = os.path.join(PROJECT_DIR, "models")
    DATA_DIR = os.path.join(PROJECT_DIR, "data")
    CACHE_DIR = os.path.join(PROJECT_DIR, "cache")

    def __post_init__(self):
        os.makedirs(self.MODEL_DIR, exist_ok=True)
        os.makedirs(self.DATA_DIR, exist_ok=True)
        os.makedirs(self.CACHE_DIR, exist_ok=True)

config = ColabConfig()

print(f"🔧 Colab Configuration:")
print(f"   Device: {config.DEVICE}")
print(f"   Embedding Dim: {config.EMBEDDING_DIM}")
print(f"   Batch Size: {config.BATCH_SIZE}")
print(f"   Project Dir: {PROJECT_DIR}")

# GPU Memory check
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"   GPU Memory: {gpu_memory:.1f} GB")

    # Clear cache
    torch.cuda.empty_cache()


In [ ]:

import io
from pathlib import Path
from typing import Dict, List
from docx import Document
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

class ColabDocumentParser:
    """Colab-optimized document parser with safe handling"""

    SUPPORTED_FORMATS = {'.pdf', '.docx', '.txt'}

    def __init__(self):
        try:
            self.nlp = spacy.load("en_core_web_sm")
            logger.info("✅ SpaCy model loaded successfully")
        except Exception as e:
            logger.warning(f"SpaCy model loading failed: {e}")
            self.nlp = None

    def parse_pdf(self, file_content: bytes) -> Dict:
        try:
            text_parts = []
            with pdfplumber.open(io.BytesIO(file_content)) as pdf:
                for page in pdf.pages[:5]:  # limit to 5 pages for memory
                    text = page.extract_text() or ""
                    if text.strip():
                        text_parts.append(text)
            full_text = "\n".join(text_parts)
            return {'text': full_text, 'word_count': len(full_text.split()), 'pages': len(text_parts), 'success': True}
        except Exception as e:
            logger.error(f"PDF parsing failed: {e}")
            return {'text': '', 'word_count': 0, 'pages': 0, 'success': False}

    def parse_docx(self, file_content: bytes) -> Dict:
        try:
            doc = Document(io.BytesIO(file_content))
            text_parts = [p.text for p in doc.paragraphs if p.text.strip()]
            full_text = "\n".join(text_parts)
            return {'text': full_text, 'word_count': len(full_text.split()), 'paragraphs': len(text_parts), 'success': True}
        except Exception as e:
            logger.error(f"DOCX parsing failed: {e}")
            return {'text': '', 'word_count': 0, 'paragraphs': 0, 'success': False}

    def parse_text(self, file_content: bytes) -> Dict:
        try:
            text = file_content.decode('utf-8')
            return {'text': text, 'word_count': len(text.split()), 'success': True}
        except Exception:
            try:
                text = file_content.decode('latin-1', errors='ignore')
                return {'text': text, 'word_count': len(text.split()), 'success': True}
            except Exception as e2:
                logger.error(f"Text parsing failed: {e2}")
                return {'text': '', 'word_count': 0, 'success': False}

    def parse_document(self, file_content: bytes, filename: str) -> Dict:
        """Main parsing method with full safety"""
        # Step 1: Sanitize filename
        clean_name = os.path.basename(filename)
        clean_name = re.sub(r'[?\\]', '', clean_name).strip()

        # Step 2: Validate extension
        file_ext = Path(clean_name).suffix.lower()
        if file_ext not in self.SUPPORTED_FORMATS:
            return {'text': '', 'word_count': 0, 'success': False, 'error': f'Unsupported file format: {file_ext}'}

        # Step 3: Parse document safely
        try:
            if file_ext == '.pdf':
                result = self.parse_pdf(file_content)
            elif file_ext == '.docx':
                result = self.parse_docx(file_content)
            elif file_ext == '.txt':
                result = self.parse_text(file_content)
            else:
                # should never reach here due to validation
                result = {'text': '', 'word_count': 0, 'success': False, 'error': 'Unknown format'}
        except Exception as e:
            return {'text': '', 'word_count': 0, 'success': False, 'error': f'Parsing failed: {e}'}

        # Step 4: Enrich result if parsing succeeded
        if result.get('success') and result.get('text'):
            result['contact'] = self.extract_contact_info(result['text'])
            result['sections'] = self._identify_sections(result['text'])
            result['filename'] = clean_name

        return result


    import re

    def extract_contact_info(self, text: str):
        contact_info = {}

        # Email
        email_match = re.search(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", text)
        if email_match:
            contact_info['email'] = email_match.group(0)

        # Phone (supports +91, +1, (123) 456-7890, etc.)
        phone_match = re.search(r"(\+?\d{1,3}[-.\s]?)?\(?\d{2,4}\)?[-.\s]?\d{3,4}[-.\s]?\d{3,4}", text)
        if phone_match:
            contact_info['phone'] = phone_match.group(0)

        # LinkedIn
        linkedin_match = re.search(r"(linkedin\.com\/in\/[A-Za-z0-9_-]+)", text, re.IGNORECASE)
        if linkedin_match:
            contact_info['linkedin'] = linkedin_match.group(0)

        return contact_info

    def _identify_sections(self, text: str) -> List[str]:
        """Identify sections safely"""
        section_patterns = {
            'contact': r'(email|phone|address|linkedin)',
            'summary': r'(summary|objective|profile)',
            'experience': r'(experience|employment|work\s+history)',
            'education': r'(education|academic|degree)',
            'skills': r'(skills|competencies|technical)',
            'projects': r'(projects|portfolio)',
            'achievements': r'(achievements|awards|honors)'
        }
        found_sections = []
        text_lower = text.lower()
        for section, pattern in section_patterns.items():
            if re.search(pattern, text_lower):
                found_sections.append(section)
        return found_sections

# Test the parser
parser = ColabDocumentParser()
print("✅ Document parser initialized safely")


In [ ]:
class ColabFeatureExtractor:
    """Colab-optimized feature extractor"""

    def __init__(self):
        # Load smaller sentence transformer for Colab
        try:
            self.sentence_transformer = SentenceTransformer(config.SENTENCE_TRANSFORMER)
            logger.info(f"✅ Sentence transformer loaded: {config.SENTENCE_TRANSFORMER}")
        except Exception as e:
            logger.error(f"Failed to load sentence transformer: {e}")
            self.sentence_transformer = None

        # Load spaCy
        try:
            self.nlp = spacy.load("en_core_web_sm")
        except:
            self.nlp = None

        # Skill database
        self.skill_database = {
            'programming': ['python', 'java', 'javascript', 'sql', 'r', 'c++', 'c#', 'php'],
            'data_science': ['machine learning', 'tensorflow', 'pytorch', 'pandas', 'numpy', 'scikit-learn'],
            'web_dev': ['react', 'angular', 'vue', 'node.js', 'html', 'css', 'bootstrap'],
            'cloud': ['aws', 'azure', 'gcp', 'docker', 'kubernetes'],
            'databases': ['mysql', 'postgresql', 'mongodb', 'redis'],
            'soft_skills': ['leadership', 'communication', 'teamwork', 'problem solving']
        }

    def extract_skills(self, text: str) -> Dict[str, List[str]]:
        """Extract skills by category"""
        text_lower = text.lower()
        found_skills = {}

        for category, skills in self.skill_database.items():
            category_skills = []
            for skill in skills:
                pattern = r'\\b' + re.escape(skill.lower()) + r'\\b'
                if re.search(pattern, text_lower):
                    category_skills.append(skill)

            if category_skills:
                found_skills[category] = category_skills

        return found_skills

    def extract_experience_years(self, text: str) -> float:
        """Extract years of experience"""
        patterns = [
            r'(\\d+)\\s*\\+?\\s*years?\\s*of\\s*experience',
            r'(\\d+)\\s*\\+?\\s*yrs?\\s*experience',
            r'experience.*?(\\d+)\\s*\\+?\\s*years?',
        ]

        years = []
        for pattern in patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            years.extend([int(match) for match in matches])

        # Also try to extract from date ranges
        date_pattern = r'\\b(20\\d{2}|19\\d{2})\\b'
        dates = [int(date) for date in re.findall(date_pattern, text)]

        calculated_years = 0
        if len(dates) >= 2:
            calculated_years = max(dates) - min(dates)

        # Return the maximum found
        all_years = years + [calculated_years]
        return max(all_years) if all_years else 0.0

    def extract_education_level(self, text: str) -> int:
        """Extract education level (0-4 scale)"""
        education_patterns = {
            4: [r'ph\\.?d\\.?', r'doctorate', r'doctoral'],
            3: [r"master's?", r'm\\.s\\.?', r'm\\.a\\.?', r'mba'],
            2: [r"bachelor's?", r'b\\.s\\.?', r'b\\.a\\.?', r'undergraduate'],
            1: [r'associate', r'a\\.s\\.?', r'a\\.a\\.?']
        }

        text_lower = text.lower()
        max_level = 0

        for level, patterns in education_patterns.items():
            for pattern in patterns:
                if re.search(pattern, text_lower):
                    max_level = max(max_level, level)

        return max_level

    def calculate_readability_score(self, text: str) -> float:
        """Calculate readability score"""
        try:
            if len(text) < 50:
                return 50.0

            flesch_score = textstat.flesch_reading_ease(text)
            return flesch_score if flesch_score is not None else 50.0
        except:
            return 50.0

    def get_text_embedding(self, text: str) -> np.ndarray:
        """Get text embedding using sentence transformer"""
        if not self.sentence_transformer or not text.strip():
            return np.zeros(config.EMBEDDING_DIM, dtype=np.float32)

        try:
            embedding = self.sentence_transformer.encode(text, convert_to_numpy=True)
            # Ensure correct dimension
            if embedding.shape[0] != config.EMBEDDING_DIM:
                # Pad or truncate to match expected dimension
                if embedding.shape[0] > config.EMBEDDING_DIM:
                    embedding = embedding[:config.EMBEDDING_DIM]
                else:
                    padded = np.zeros(config.EMBEDDING_DIM)
                    padded[:embedding.shape[0]] = embedding
                    embedding = padded

            return embedding.astype(np.float32)
        except Exception as e:
            logger.warning(f"Embedding failed: {e}")
            return np.zeros(config.EMBEDDING_DIM, dtype=np.float32)

    def extract_all_features(self, document: Dict, job_description: str = "") -> Dict:
        """Extract comprehensive features from document"""
        text = document.get('text', '')

        if not text.strip():
            return self._get_empty_features()

        # Extract various features
        skills = self.extract_skills(text)
        experience_years = self.extract_experience_years(text)
        education_level = self.extract_education_level(text)
        readability = self.calculate_readability_score(text)

        # Calculate derived metrics
        total_skills = sum(len(skill_list) for skill_list in skills.values())
        skill_diversity = len(skills.keys()) / 6.0 if skills else 0.0  # Max 6 categories

        # Contact completeness
        contact = document.get('contact', {})
        contact_score = 0.0
        if contact.get('email'): contact_score += 0.4
        if contact.get('phone'): contact_score += 0.4
        if contact.get('linkedin'): contact_score += 0.1
        if contact.get('github'): contact_score += 0.1

        # Section completeness
        sections = document.get('sections', [])
        required_sections = ['experience', 'education', 'skills']
        section_score = len(set(sections) & set(required_sections)) / len(required_sections)

        # ATS friendliness
        word_count = document.get('word_count', 0)
        ats_score = 0.0
        if word_count > 200: ats_score += 0.3
        if contact.get('email'): ats_score += 0.2
        ats_score += section_score * 0.3
        ats_score += min(1.0, word_count / 1000) * 0.2

        # Keyword density (if job description provided)
        keyword_density = 0.0
        if job_description:
            job_words = set(job_description.lower().split())
            resume_words = set(text.lower().split())
            common_words = job_words & resume_words
            keyword_density = len(common_words) / max(len(job_words), 1)

        # Get embeddings
        doc_embedding = self.get_text_embedding(text)

        # Section embeddings (simplified for Colab)
        section_embeddings = {}
        for section in ['experience', 'education', 'skills']:
            if section in text.lower():
                # Extract section text (simplified)
                section_start = text.lower().find(section)
                section_text = text[section_start:section_start+500] if section_start >= 0 else ""
                section_embeddings[section] = self.get_text_embedding(section_text)
            else:
                section_embeddings[section] = np.zeros(config.EMBEDDING_DIM, dtype=np.float32)

        return {
            # Basic metrics
            'word_count': word_count,
            'section_count': len(sections),
            'experience_years': experience_years,
            'education_level': education_level,
            'total_skills': total_skills,

            # Calculated scores
            'skill_diversity': skill_diversity,
            'contact_completeness': contact_score,
            'section_completeness': section_score,
            'ats_friendliness': ats_score,
            'readability_score': readability / 100.0,  # Normalize to 0-1
            'keyword_density': keyword_density,

            # Binary features
            'has_email': 1.0 if contact.get('email') else 0.0,
            'has_phone': 1.0 if contact.get('phone') else 0.0,
            'has_linkedin': 1.0 if contact.get('linkedin') else 0.0,
            'has_achievements': 1.0 if 'award' in text.lower() or 'achievement' in text.lower() else 0.0,

            # Embeddings
            'document_embedding': doc_embedding,
            'experience_embedding': section_embeddings.get('experience', np.zeros(config.EMBEDDING_DIM)),
            'education_embedding': section_embeddings.get('education', np.zeros(config.EMBEDDING_DIM)),
            'skills_embedding': section_embeddings.get('skills', np.zeros(config.EMBEDDING_DIM)),

            # Additional info
            'skills_found': skills,
            'sections_found': sections,
            'contact_info': contact
        }

    def _get_empty_features(self) -> Dict:
        """Return empty features for failed parsing"""
        return {
            'word_count': 0, 'section_count': 0, 'experience_years': 0.0,
            'education_level': 0, 'total_skills': 0, 'skill_diversity': 0.0,
            'contact_completeness': 0.0, 'section_completeness': 0.0,
            'ats_friendliness': 0.0, 'readability_score': 0.5, 'keyword_density': 0.0,
            'has_email': 0.0, 'has_phone': 0.0, 'has_linkedin': 0.0, 'has_achievements': 0.0,
            'document_embedding': np.zeros(config.EMBEDDING_DIM),
            'experience_embedding': np.zeros(config.EMBEDDING_DIM),
            'education_embedding': np.zeros(config.EMBEDDING_DIM),
            'skills_embedding': np.zeros(config.EMBEDDING_DIM),
            'skills_found': {}, 'sections_found': [], 'contact_info': {}
        }

# Initialize feature extractor
extractor = ColabFeatureExtractor()
print("✅ Feature extractor initialized")


In [ ]:
class ColabResumeScorer(nn.Module):
    """Colab-optimized resume scoring model"""

    def __init__(self,
                 embedding_dim: int = config.EMBEDDING_DIM,
                 hidden_dims: List[int] = config.HIDDEN_DIMS,
                 dropout_rates: List[float] = config.DROPOUT_RATES,
                 num_attention_heads: int = 4):  # Reduced for Colab
        super().__init__()

        self.embedding_dim = embedding_dim

        # Input projections for different embeddings
        self.document_proj = nn.Linear(embedding_dim, hidden_dims[0])
        self.experience_proj = nn.Linear(embedding_dim, hidden_dims[0])
        self.education_proj = nn.Linear(embedding_dim, hidden_dims[0])
        self.skills_proj = nn.Linear(embedding_dim, hidden_dims[0])

        # Numeric features processor (14 numeric features)
        self.numeric_features_dim = 14
        self.numeric_proj = nn.Sequential(
            nn.Linear(self.numeric_features_dim, hidden_dims[0] // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rates[0])
        )

        # Multi-head attention for feature fusion
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_dims[0],
            num_heads=num_attention_heads,
            dropout=dropout_rates[0],
            batch_first=True
        )

        # Feature fusion layers
        fusion_input_dim = hidden_dims[0] * 4 + hidden_dims[0] // 2

        self.fusion_layers = nn.Sequential(
            nn.Linear(fusion_input_dim, hidden_dims[1]),
            nn.LayerNorm(hidden_dims[1]),
            nn.ReLU(),
            nn.Dropout(dropout_rates[1]),

            nn.Linear(hidden_dims[1], hidden_dims[2]),
            nn.LayerNorm(hidden_dims[2]),
            nn.ReLU(),
            nn.Dropout(dropout_rates[2])
        )

        # Output heads
        final_dim = hidden_dims[2]

        self.skills_head = nn.Sequential(
            nn.Linear(final_dim, final_dim // 2),
            nn.ReLU(),
            nn.Linear(final_dim // 2, 1),
            nn.Sigmoid()
        )

        self.experience_head = nn.Sequential(
            nn.Linear(final_dim, final_dim // 2),
            nn.ReLU(),
            nn.Linear(final_dim // 2, 1),
            nn.Sigmoid()
        )

        self.education_head = nn.Sequential(
            nn.Linear(final_dim, final_dim // 2),
            nn.ReLU(),
            nn.Linear(final_dim // 2, 1),
            nn.Sigmoid()
        )

        self.formatting_head = nn.Sequential(
            nn.Linear(final_dim, final_dim // 2),
            nn.ReLU(),
            nn.Linear(final_dim // 2, 1),
            nn.Sigmoid()
        )

        self.achievements_head = nn.Sequential(
            nn.Linear(final_dim, final_dim // 2),
            nn.ReLU(),
            nn.Linear(final_dim // 2, 1),
            nn.Sigmoid()
        )

        # Overall score head
        self.overall_head = nn.Sequential(
            nn.Linear(final_dim, final_dim // 2),
            nn.ReLU(),
            nn.Linear(final_dim // 2, 1),
            nn.Sigmoid()
        )

        # Initialize weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.ones_(module.weight)
            torch.nn.init.zeros_(module.bias)

    def forward(self, embeddings_dict: Dict[str, torch.Tensor], numeric_features: torch.Tensor):
        batch_size = numeric_features.shape[0]

        # Project embeddings
        doc_emb = self.document_proj(embeddings_dict['document'])
        exp_emb = self.experience_proj(embeddings_dict['experience'])
        edu_emb = self.education_proj(embeddings_dict['education'])
        skills_emb = self.skills_proj(embeddings_dict['skills'])

        # Process numeric features
        numeric_features = torch.nan_to_num(numeric_features, nan=0.0)
        numeric_emb = self.numeric_proj(numeric_features)

        # Stack embeddings for attention
        embedding_stack = torch.stack([doc_emb, exp_emb, edu_emb, skills_emb], dim=1)

        # Apply attention
        attended_emb, attention_weights = self.attention(
            embedding_stack, embedding_stack, embedding_stack
        )
        attended_emb = attended_emb.mean(dim=1)  # Pool

        # Concatenate all features
        fused_features = torch.cat([
            doc_emb, exp_emb, edu_emb, skills_emb, numeric_emb
        ], dim=1)

        # Pass through fusion layers
        x = self.fusion_layers(fused_features)

        # Generate scores
        scores = {
            'skills': self.skills_head(x).squeeze(-1),
            'experience': self.experience_head(x).squeeze(-1),
            'education': self.education_head(x).squeeze(-1),
            'formatting': self.formatting_head(x).squeeze(-1),
            'achievements': self.achievements_head(x).squeeze(-1),
            'overall': self.overall_head(x).squeeze(-1)
        }

        return scores, attention_weights

print("✅ Deep learning model defined")


In [ ]:
class ColabResumeDataset(Dataset):

    def __init__(self, features_list: List[Dict], scores_list: List[Dict],
                 scaler: Optional[StandardScaler] = None, augment: bool = False):

        self.features_list = features_list
        self.scores_list = scores_list

        # Numeric feature keys (14 features)
        self.numeric_keys = [
            'word_count', 'section_count', 'experience_years', 'education_level',
            'total_skills', 'skill_diversity', 'contact_completeness', 'section_completeness',
            'ats_friendliness', 'readability_score', 'keyword_density',
            'has_email', 'has_phone', 'has_linkedin'
        ]

        # Build numeric matrix
        self.numeric_matrix, self.scaler = self._build_numeric_matrix(scaler)

        # Process embeddings
        self.embeddings_dict = self._process_embeddings()

        # Process targets
        self.targets = self._process_targets()

        # Apply augmentation if requested
        if augment:
            self._apply_colab_augmentation()

        logger.info(f"Dataset initialized: {len(self)} samples")

    def _build_numeric_matrix(self, scaler):
        numeric_data = []

        for features in self.features_list:
            row = []
            for key in self.numeric_keys:
                value = features.get(key, 0.0)
                # Handle special cases
                if key == 'word_count':
                    value = min(value, 2000)  # Cap at 2000
                elif key == 'experience_years':
                    value = min(value, 20)  # Cap at 20 years
                elif key == 'total_skills':
                    value = min(value, 30)  # Cap at 30 skills

                row.append(float(value))
            numeric_data.append(row)

        numeric_matrix = np.array(numeric_data, dtype=np.float32)
        numeric_matrix = np.nan_to_num(numeric_matrix, nan=0.0)

        # Scale features
        if scaler is None:
            scaler = StandardScaler()
            numeric_matrix = scaler.fit_transform(numeric_matrix)
        else:
            numeric_matrix = scaler.transform(numeric_matrix)

        return numeric_matrix.astype(np.float32), scaler

    def _process_embeddings(self):
        embedding_types = ['document', 'experience', 'education', 'skills']
        embeddings_dict = {}

        for emb_type in embedding_types:
            embeddings = []
            for features in self.features_list:
                key = f'{emb_type}_embedding'
                emb = features.get(key, np.zeros(config.EMBEDDING_DIM))

                # Ensure correct shape
                if emb.shape[0] != config.EMBEDDING_DIM:
                    emb = np.zeros(config.EMBEDDING_DIM)

                embeddings.append(emb)

            embeddings_dict[emb_type] = np.array(embeddings, dtype=np.float32)

        return embeddings_dict

    def _process_targets(self):
        targets = []
        target_keys = ['skills', 'experience', 'education', 'formatting', 'achievements', 'overall']

        for scores in self.scores_list:
            target_row = []
            for key in target_keys:
                value = scores.get(key, 0.5)  # Default to middle score
                value = max(0.0, min(1.0, value))  # Clamp to [0, 1]
                target_row.append(value)
            targets.append(target_row)

        return np.array(targets, dtype=np.float32)

    def _apply_colab_augmentation(self):
        """Light augmentation for Colab memory constraints"""
        original_size = len(self.features_list)

        # Only augment a subset to save memory
        augment_size = min(original_size, 200)  # Limit augmentation

        for i in range(augment_size):
            # Add noise to numeric features
            noisy_numeric = self.numeric_matrix[i] + np.random.normal(0, 0.05, self.numeric_matrix[i].shape)
            self.numeric_matrix = np.vstack([self.numeric_matrix, noisy_numeric])

            # Add noise to embeddings
            for emb_type in self.embeddings_dict:
                noisy_emb = self.embeddings_dict[emb_type][i] + np.random.normal(0, 0.02, self.embeddings_dict[emb_type][i].shape)
                self.embeddings_dict[emb_type] = np.vstack([self.embeddings_dict[emb_type], noisy_emb])

            # Copy targets with small noise
            noisy_target = self.targets[i] + np.random.normal(0, 0.01, self.targets[i].shape)
            noisy_target = np.clip(noisy_target, 0, 1)
            self.targets = np.vstack([self.targets, noisy_target])

        logger.info(f"Augmented dataset from {original_size} to {len(self.targets)} samples")

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        embeddings = {}
        for emb_type in self.embeddings_dict:
            embeddings[emb_type] = torch.tensor(self.embeddings_dict[emb_type][idx], dtype=torch.float32)

        numeric = torch.tensor(self.numeric_matrix[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)

        return {'embeddings': embeddings, 'numeric': numeric, 'target': target}

class ColabTrainer:
    """Colab-optimized trainer with memory management"""

    def __init__(self, model: nn.Module, device=config.DEVICE):
        self.model = model.to(device)
        self.device = device
        self.optimizer = optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, patience=3, factor=0.5)
        self.criterion = nn.MSELoss()

        # Training history
        self.train_losses = []
        self.val_losses = []

    def compute_loss(self, predictions, targets):
        target_keys = ['skills', 'experience', 'education', 'formatting', 'achievements', 'overall']
        total_loss = 0.0

        for i, key in enumerate(target_keys):
            if key in predictions:
                loss = self.criterion(predictions[key], targets[:, i])
                total_loss += loss

        return total_loss / len(target_keys)

    def train_epoch(self, train_loader):
        self.model.train()
        total_loss = 0.0
        num_batches = 0

        for batch in tqdm(train_loader, desc="Training", leave=False):
            # Clear gradients and cache
            self.optimizer.zero_grad()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            # Move to device
            embeddings = {k: v.to(self.device) for k, v in batch['embeddings'].items()}
            numeric = batch['numeric'].to(self.device)
            targets = batch['target'].to(self.device)

            # Forward pass
            predictions, _ = self.model(embeddings, numeric)
            loss = self.compute_loss(predictions, targets)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()

            total_loss += loss.item()
            num_batches += 1

        return total_loss / num_batches

    def validate_epoch(self, val_loader):
        self.model.eval()
        total_loss = 0.0
        num_batches = 0

        with torch.no_grad():
            for batch in val_loader:
                embeddings = {k: v.to(self.device) for k, v in batch['embeddings'].items()}
                numeric = batch['numeric'].to(self.device)
                targets = batch['target'].to(self.device)

                predictions, _ = self.model(embeddings, numeric)
                loss = self.compute_loss(predictions, targets)

                total_loss += loss.item()
                num_batches += 1

        return total_loss / num_batches

    def fit(self, train_loader, val_loader, epochs=config.EPOCHS):
        logger.info(f"Starting training for {epochs} epochs...")

        best_val_loss = float('inf')
        patience_counter = 0

        for epoch in range(epochs):
            # Training
            train_loss = self.train_epoch(train_loader)
            val_loss = self.validate_epoch(val_loader)

            # Update scheduler
            self.scheduler.step(val_loss)

            # Store history
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)

            # Print progress
            if epoch % 2 == 0:  # Print every 2 epochs
                logger.info(f"Epoch {epoch+1}/{epochs} - Train: {train_loss:.4f}, Val: {val_loss:.4f}")

            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # Save best model
                torch.save(self.model.state_dict(), os.path.join(config.MODEL_DIR, "best_model.pth"))
            else:
                patience_counter += 1
                if patience_counter >= 5:  # Early stopping
                    logger.info("Early stopping triggered")
                    break

        # Load best model
        self.model.load_state_dict(torch.load(os.path.join(config.MODEL_DIR, "best_model.pth")))
        logger.info(f"Training completed. Best validation loss: {best_val_loss:.4f}")

        return self.train_losses, self.val_losses

print("✅ Dataset and trainer classes defined")



In [ ]:

def load_kaggle_resume_data(max_samples=500):
    """Load resume data from Kaggle with Colab optimization"""
    try:
        # Download dataset
        logger.info("Downloading Kaggle resume dataset...")
        path = kagglehub.dataset_download("gauravduttakiit/resume-dataset")
        df = pd.read_csv(os.path.join(path, "UpdatedResumeDataSet.csv"))

        # Sample data for Colab memory constraints
        if len(df) > max_samples:
            df = df.sample(n=max_samples, random_state=42)

        logger.info(f"Loaded {len(df)} resumes")

        return df

    except Exception as e:
        logger.error(f"Failed to load Kaggle data: {e}")
        # Return sample data if Kaggle fails
        return create_sample_data(max_samples)

def create_sample_data(num_samples=100):
    """Create sample resume data for testing"""
    logger.info(f"Creating {num_samples} sample resumes...")

    sample_resumes = []
    categories = ['Software Engineering', 'Data Science', 'Marketing', 'Sales', 'Design']

    for i in range(num_samples):
        category = np.random.choice(categories)
        years_exp = np.random.randint(1, 15)

        # Generate sample resume text
        resume_text = f"""
        John Doe {i}
        Email: john{i}@email.com
        Phone: (555) 123-{i:04d}

        PROFESSIONAL SUMMARY
        {category} professional with {years_exp} years of experience.

        EXPERIENCE
        Senior {category.split()[0]} - TechCorp Inc. (2020-2024)
        • Led team of {np.random.randint(3, 10)} professionals
        • Increased efficiency by {np.random.randint(10, 50)}%
        • Managed projects worth ${np.random.randint(100, 1000)}K
        • Implemented {np.random.choice(['Python', 'JavaScript', 'SQL', 'React'])} solutions

        EDUCATION
        {np.random.choice(['Bachelor of Science', 'Master of Science'])} in {category.split()[0]}
        State University - Graduated {2024 - years_exp + 2}

        SKILLS
        Technical: {', '.join(np.random.choice(['Python', 'Java', 'SQL', 'AWS', 'Docker', 'React'], 4))}
        Soft Skills: Leadership, Communication, Problem Solving

        ACHIEVEMENTS
        • Won {np.random.choice(['Excellence', 'Innovation', 'Leadership'])} Award
        • Published {np.random.randint(1, 5)} papers/articles
        • Certified in {np.random.choice(['AWS', 'Google Cloud', 'Microsoft Azure'])}
        """

        sample_resumes.append({
            'Resume': resume_text.strip(),
            'Category': category
        })

    return pd.DataFrame(sample_resumes)

def generate_realistic_scores(text, category):
    """Generate realistic scores based on resume content"""
    word_count = len(text.split())

    # Base scores on content analysis
    skills_keywords = ['python', 'java', 'sql', 'aws', 'react', 'machine learning', 'leadership']
    skills_found = sum(1 for skill in skills_keywords if skill.lower() in text.lower())

    # Generate scores with some randomness but based on content
    skills_score = min(0.95, (skills_found / len(skills_keywords)) * 0.7 + 0.2 + np.random.normal(0, 0.1))
    experience_score = min(0.95, (word_count / 600) * 0.6 + 0.25 + np.random.normal(0, 0.1))
    education_score = min(0.95, 0.5 + np.random.normal(0, 0.15))
    formatting_score = min(0.95, 0.6 + np.random.normal(0, 0.1))
    achievements_score = min(0.95, 0.4 + np.random.normal(0, 0.15))

    # Overall score as weighted average
    overall_score = (
        skills_score * 0.25 +
        experience_score * 0.25 +
        education_score * 0.15 +
        formatting_score * 0.15 +
        achievements_score * 0.20
    )

    return {
        'skills': max(0.1, min(0.95, skills_score)),
        'experience': max(0.1, min(0.95, experience_score)),
        'education': max(0.1, min(0.95, education_score)),
        'formatting': max(0.1, min(0.95, formatting_score)),
        'achievements': max(0.1, min(0.95, achievements_score)),
        'overall': max(0.1, min(0.95, overall_score))
    }

def process_resume_dataset(df, max_samples=None):
    """Process resume dataset with Colab optimization"""
    if max_samples and len(df) > max_samples:
        df = df.sample(n=max_samples, random_state=42)

    features_list = []
    scores_list = []

    logger.info("Processing resumes...")
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
        try:
            # Create document structure
            text = str(row.get('Resume', ''))
            category = str(row.get('Category', 'Unknown'))

            # Mock document for feature extraction
            document = {
                'text': text,
                'word_count': len(text.split()),
                'success': True,
                'contact': {'email': 'test@email.com'},  # Mock contact
                'sections': ['experience', 'education', 'skills']  # Mock sections
            }

            # Extract features
            features = extractor.extract_all_features(document)

            # Generate realistic scores
            scores = generate_realistic_scores(text, category)

            features_list.append(features)
            scores_list.append(scores)

        except Exception as e:
            logger.warning(f"Failed to process resume {idx}: {e}")
            continue

    logger.info(f"Successfully processed {len(features_list)} resumes")
    return features_list, scores_list

print("✅ Data loading functions defined")


In [ ]:
# ============================================================================
# 📱 CELL 8: Training Pipeline
# ============================================================================

def train_colab_model(max_samples=300, epochs=15, batch_size=6):
    """Complete training pipeline optimized for Colab"""

    logger.info("🚀 Starting Colab Resume Analyzer Training")
    logger.info("=" * 50)

    # Step 1: Load data
    logger.info("📊 Step 1: Loading data...")
    df = load_kaggle_resume_data(max_samples)

    # Step 2: Process resumes
    logger.info("🔄 Step 2: Processing resumes...")
    features_list, scores_list = process_resume_dataset(df, max_samples)

    if len(features_list) == 0:
        raise ValueError("No resumes processed successfully!")

    # Step 3: Create dataset
    logger.info("📦 Step 3: Creating dataset...")
    dataset = ColabResumeDataset(features_list, scores_list, augment=True)

    # Step 4: Split data
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    # Step 5: Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    logger.info(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

    # Step 6: Initialize model
    logger.info("🧠 Step 4: Initializing model...")
    model = ColabResumeScorer()

    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    logger.info(f"Model parameters: {total_params:,}")

    # Step 7: Train model
    logger.info("🏋️ Step 5: Training model...")
    trainer = ColabTrainer(model)
    train_losses, val_losses = trainer.fit(train_loader, val_loader, epochs=epochs)

    # Step 8: Save components
    logger.info("💾 Step 6: Saving model and scaler...")
    torch.save(model.state_dict(), os.path.join(config.MODEL_DIR, "colab_model.pth"))

    with open(os.path.join(config.MODEL_DIR, "scaler.pkl"), 'wb') as f:
        pickle.dump(dataset.scaler, f)

    # Step 9: Plot training history
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Training History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(val_losses)
    plt.title('Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.tight_layout()
    plt.show()

    logger.info("✅ Training completed successfully!")
    logger.info(f"📈 Final validation loss: {min(val_losses):.4f}")

    return model, dataset, trainer

print("✅ Training pipeline defined")


✅ Training pipeline defined


In [ ]:
# ============================================================================
# 📱 CELL 9: Production Analyzer
# ============================================================================

class ColabResumeAnalyzer:
    """Production-ready resume analyzer for Colab"""

    def __init__(self, model_path=None, scaler_path=None):
        self.device = config.DEVICE
        self.parser = ColabDocumentParser()
        self.extractor = ColabFeatureExtractor()

        # Load model
        if model_path and os.path.exists(model_path):
            self.model = ColabResumeScorer()
            self.model.load_state_dict(torch.load(model_path, map_location=self.device))
            self.model.eval()
            logger.info(f"✅ Model loaded from {model_path}")
        else:
            logger.warning("⚠️ No model loaded - predictions will be random")
            self.model = ColabResumeScorer()

        # Load scaler
        if scaler_path and os.path.exists(scaler_path):
            with open(scaler_path, 'rb') as f:
                self.scaler = pickle.load(f)
            logger.info(f"✅ Scaler loaded from {scaler_path}")
        else:
            logger.warning("⚠️ No scaler loaded")
            self.scaler = None

    def analyze(self, file_path, job_description=""):
        """Analyze resume from file path"""
        try:
            with open(file_path, 'rb') as f:
                file_content = f.read()
            filename = os.path.basename(file_path)
            return self.analyze_resume_bytes(file_content, filename, job_description)
        except Exception as e:
            return {'success': False, 'error': f'File reading failed: {e}'}

    def analyze_resume_bytes(self, file_content, filename, job_description=""):
        """Analyze resume from bytes"""
        try:
            # Step 1: Parse document
            document = self.parser.parse_document(file_content, filename)

            if not document.get('success', False):
                return {
                    'success': False,
                    'error': 'Document parsing failed',
                    'details': document.get('error', 'Unknown parsing error')
                }

            # Step 2: Extract features
            features = self.extractor.extract_all_features(document, job_description)

            # Step 3: Prepare model inputs
            model_inputs = self._prepare_model_inputs(features)

            # Step 4: Get predictions
            with torch.no_grad():
                predictions, attention_weights = self.model(
                    model_inputs['embeddings'],
                    model_inputs['numeric']
                )

            # Step 5: Convert to scores (0-100)
            scores = {}
            for key, value in predictions.items():
                scores[key] = float(value.cpu().item() * 100)

            # Step 6: Generate analysis
            analysis = self._generate_analysis(features, scores, document)

            # Step 7: Generate recommendations
            recommendations = self._generate_recommendations(features, scores, job_description)

            return {
                'success': True,
                'filename': filename,
                'scores': scores,
                'analysis': analysis,
                'recommendations': recommendations,
                'document_info': {
                    'word_count': document.get('word_count', 0),
                    'sections_found': document.get('sections', []),
                    'contact_info': document.get('contact', {})
                }
            }

        except Exception as e:
            logger.error(f"Analysis failed: {e}")
            return {
                'success': False,
                'error': f'Analysis failed: {str(e)}'
            }

    def _prepare_model_inputs(self, features):
        """Prepare features for model input"""
        # Extract embeddings
        embeddings = {}
        for emb_type in ['document', 'experience', 'education', 'skills']:
            key = f'{emb_type}_embedding'
            emb = features.get(key, np.zeros(config.EMBEDDING_DIM))
            embeddings[emb_type] = torch.tensor(emb, dtype=torch.float32).unsqueeze(0).to(self.device)

        # Extract numeric features
        numeric_keys = [
            'word_count', 'section_count', 'experience_years', 'education_level',
            'total_skills', 'skill_diversity', 'contact_completeness', 'section_completeness',
            'ats_friendliness', 'readability_score', 'keyword_density',
            'has_email', 'has_phone', 'has_linkedin'
        ]

        numeric_values = []
        for key in numeric_keys:
            value = features.get(key, 0.0)
            # Apply same caps as in training
            if key == 'word_count':
                value = min(value, 2000)
            elif key == 'experience_years':
                value = min(value, 20)
            elif key == 'total_skills':
                value = min(value, 30)
            numeric_values.append(float(value))

        numeric_tensor = torch.tensor(numeric_values, dtype=torch.float32).unsqueeze(0)

        # Scale if scaler available
        if self.scaler:
            numeric_np = numeric_tensor.numpy()
            numeric_scaled = self.scaler.transform(numeric_np)
            numeric_tensor = torch.tensor(numeric_scaled, dtype=torch.float32)

        numeric_tensor = numeric_tensor.to(self.device)

        return {'embeddings': embeddings, 'numeric': numeric_tensor}

    def _generate_analysis(self, features, scores, document):
        """Generate comprehensive analysis"""
        analysis = {
            'strengths': [],
            'weaknesses': [],
            'key_metrics': {}
        }

        # Identify strengths and weaknesses
        for component, score in scores.items():
            if component == 'overall':
                continue

            if score >= 80:
                analysis['strengths'].append(f"Excellent {component} section (Score: {score:.1f})")
            elif score >= 65:
                analysis['strengths'].append(f"Good {component} presentation (Score: {score:.1f})")
            elif score < 50:
                analysis['weaknesses'].append(f"{component.title()} section needs improvement (Score: {score:.1f})")
            elif score < 65:
                analysis['weaknesses'].append(f"{component.title()} could be enhanced (Score: {score:.1f})")

        # Key metrics
        analysis['key_metrics'] = {
            'word_count': features.get('word_count', 0),
            'experience_years': features.get('experience_years', 0),
            'total_skills': features.get('total_skills', 0),
            'ats_friendliness': f"{features.get('ats_friendliness', 0) * 100:.1f}%",
            'sections_found': len(features.get('sections_found', [])),
            'contact_complete': features.get('contact_completeness', 0) > 0.5
        }

        return analysis

    def _generate_recommendations(self, features, scores, job_description):
        """Generate actionable recommendations"""
        recommendations = []

        # Skills recommendations
        if scores.get('skills', 0) < 70:
            recommendations.append({
                'category': 'Skills',
                'priority': 'HIGH',
                'action': 'Enhance technical skills section',
                'details': [
                    'Add more relevant programming languages',
                    'Include industry-specific tools',
                    'Group skills by category',
                    'Add proficiency levels'
                ]
            })

        # Experience recommendations
        if scores.get('experience', 0) < 70:
            recommendations.append({
                'category': 'Experience',
                'priority': 'HIGH',
                'action': 'Improve experience descriptions',
                'details': [
                    'Use action verbs to start bullet points',
                    'Quantify achievements with numbers',
                    'Show career progression',
                    'Include relevant keywords'
                ]
            })

        # Education recommendations
        if scores.get('education', 0) < 60:
            recommendations.append({
                'category': 'Education',
                'priority': 'MEDIUM',
                'action': 'Strengthen education section',
                'details': [
                    'Include relevant coursework',
                    'Add academic achievements',
                    'Mention certifications',
                    'Include graduation honors'
                ]
            })

        # Formatting recommendations
        if scores.get('formatting', 0) < 65:
            recommendations.append({
                'category': 'Formatting',
                'priority': 'MEDIUM',
                'action': 'Improve ATS compatibility',
                'details': [
                    'Use standard section headers',
                    'Avoid complex formatting',
                    'Ensure text is selectable',
                    'Use consistent formatting'
                ]
            })

        # Job matching
        if job_description and features.get('keyword_density', 0) < 0.2:
            recommendations.append({
                'category': 'Job Matching',
                'priority': 'HIGH',
                'action': 'Better align with job requirements',
                'details': [
                    'Include more job-specific keywords',
                    'Highlight relevant experience',
                    'Customize skills for this role',
                    'Match required qualifications'
                ]
            })

        return recommendations

    def get_ats_grade(self, overall_score):
        """Get ATS compatibility grade"""
        if overall_score >= 90:
            return "A - Excellent"
        elif overall_score >= 80:
            return "B - Very Good"
        elif overall_score >= 70:
            return "C - Good"
        elif overall_score >= 60:
            return "D - Fair"
        else:
            return "F - Needs Improvement"

print("✅ Production analyzer defined")

In [ ]:
# ============================================================================
# 📱 CELL 10: Demo Functions for Colab
# ============================================================================

def demo_file_upload():
    """Demo function for file upload in Colab"""
    if not IN_COLAB:
        print("This function is designed for Google Colab")
        return None

    print("📁 Please upload your resume file (PDF, DOCX, or TXT)")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded")
        return None

    filename = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {filename}")
    return filename

import os
import re

import pathlib
import os
import re

def sanitize_filename(filename: str) -> str:
    # Keep only alphanumeric, dash, underscore, and dot
    safe = re.sub(r'[^A-Za-z0-9._-]', '_', filename)
    return safe

def demo_analysis(analyzer, filename: str, job_desc: str = None):
    try:
        # Ensure absolute path
        filename = os.path.abspath(filename)

        # Safe name for display/logging only
        display_name = re.sub(r'[?\\]', '', os.path.basename(filename))

        print(f"🔍 Analyzing resume: {display_name}")
        print("=" * 50)

        # Pass job description if available
        if job_desc:
            result = analyzer.analyze(filename, job_description=job_desc)
        else:
            result = analyzer.analyze(filename)

        if not result['success']:
            print(f"❌ Analysis failed: {result.get('error', 'Unknown error')}")
            return result

        # Display results
        overall_score = result['scores']['overall']
        grade = analyzer.get_ats_grade(overall_score)
        print(f"🎯 Overall ATS Score: {overall_score:.1f}/100 ({grade})\n")

        print("📈 Component Scores:")
        for component, score in result['scores'].items():
            if component != 'overall':
                print(f"   {component.title()}: {score:.1f}/100")
        print()

        info = result['document_info']
        print("📄 Document Information:")
        print(f"   Word Count: {info['word_count']}")
        print(f"   Sections Found: {', '.join(info['sections_found'])}")
        print(f"   Contact Info: {list(info['contact_info'].keys())}\n")

        strengths = result['analysis']['strengths']
        if strengths:
            print("✅ Strengths:")
            for strength in strengths[:3]:
                print(f"   • {strength}")
            print()

        weaknesses = result['analysis']['weaknesses']
        if weaknesses:
            print("⚠️ Areas for Improvement:")
            for weakness in weaknesses[:3]:
                print(f"   • {weakness}")
            print()

        recommendations = result['recommendations']
        if recommendations:
            print("💡 Recommendations:")
            for i, rec in enumerate(recommendations[:2], 1):
                print(f"   {i}. {rec['action']} ({rec['priority']} Priority)")
                for detail in rec['details'][:2]:
                    print(f"      - {detail}")
            print()

        print("✅ Analysis complete!")
        return result

    except Exception as e:
        print(f"❌ Analysis failed: {e}")
        return None




def create_sample_resume_file():
    """Create a sample resume file for testing"""
    sample_resume = """
John Doe
Software Engineer
Email: john.doe@email.com
Phone: (555) 123-4567
LinkedIn: linkedin.com/in/johndoe

PROFESSIONAL SUMMARY
Experienced Software Engineer with 5 years of expertise in Python, JavaScript, and cloud technologies.
Passionate about building scalable applications and leading development teams.

WORK EXPERIENCE
Senior Software Engineer - TechCorp Inc. (2021-2024)
• Led development of microservices architecture serving 1M+ users
• Improved application performance by 40% through code optimization
• Managed team of 6 developers using Agile methodologies
• Technologies: Python, React, AWS, Docker, PostgreSQL

Software Developer - StartupXYZ (2019-2021)
• Developed full-stack web applications using Python and JavaScript
• Implemented CI/CD pipelines reducing deployment time by 60%
• Collaborated with cross-functional teams to deliver features
• Technologies: Django, React, MySQL, Jenkins

EDUCATION
Bachelor of Science in Computer Science
State University - Graduated 2019
GPA: 3.7/4.0
Relevant Coursework: Data Structures, Algorithms, Database Systems

TECHNICAL SKILLS
Programming Languages: Python, JavaScript, Java, SQL
Frameworks: Django, React, Node.js, Express
Cloud & DevOps: AWS, Docker, Kubernetes, Jenkins
Databases: PostgreSQL, MySQL, MongoDB
Tools: Git, Jira, Slack

ACHIEVEMENTS
• AWS Certified Solutions Architect
• Published 3 technical articles on Medium
• Winner of company hackathon 2022
• Mentored 8 junior developers

PROJECTS
E-commerce Platform (2023)
• Built scalable e-commerce platform handling 10K+ daily transactions
• Technologies: Python, React, AWS, PostgreSQL

Task Management App (2022)
• Developed real-time collaboration tool for remote teams
• Technologies: Node.js, React, Socket.io, MongoDB
"""

    # Save to file
    filename = "sample_resume.txt"
    with open(filename, 'w') as f:
        f.write(sample_resume)

    print(f"✅ Created sample resume: {filename}")
    return filename

print("✅ Demo functions defined")


In [ ]:

# ============================================================================
# 📱 CELL 12: Main Execution & Testing
# ============================================================================

def main_colab_demo():
    """Main demo function for Colab"""
    print("🎉 Welcome to AI Resume Analyzer - Colab Edition!")
    print("=" * 60)

    # Step 1: Train model (if not exists)
    model_path = os.path.join(config.MODEL_DIR, "colab_model.pth")
    if not os.path.exists(model_path):
        print("🏋️ No trained model found. Starting training...")
        try:
            model, dataset, trainer = train_colab_model(max_samples=200, epochs=10, batch_size=4)
            print("✅ Training completed!")
        except Exception as e:
            print(f"❌ Training failed: {e}")
            return
    else:
        print("✅ Found existing trained model")

    # Step 2: Initialize analyzer
    print("\n🔧 Initializing analyzer...")
    scaler_path = os.path.join(config.MODEL_DIR, "scaler.pkl")
    global analyzer
    analyzer = ColabResumeAnalyzer(model_path, scaler_path)

    # Step 3: Create sample resume for demo
    print("\n📄 Creating sample resume...")
    sample_file = create_sample_resume_file()

    # Step 4: Demo analysis
    print("\n🔍 Running demo analysis...")
    job_desc = "Software Engineer with Python, AWS, and React experience. Looking for someone with 3+ years experience in web development."

    result = demo_analysis(analyzer, sample_file, job_desc)

    # Step 5: Show next steps
    print("\n🚀 Next Steps:")
    print("1. Upload your own resume using demo_file_upload()")
    print("2. Analyze it with demo_analysis(analyzer, filename)")
    print("3. Start the API server with run_colab_api()")
    print("4. Train on more data with train_colab_model()")

    return analyzer

# Quick test function
def quick_test():
    """Quick test of all components"""
    print("🧪 Running quick component test...")

    # Test parser
    sample_text = "John Doe\nSoftware Engineer\njohn@email.com\nPython, Java, AWS# Google Colab Compatible - Enhanced AI Resume Analyzer"
# Run this in Google Colab for best experience


In [ ]:
train_colab_model()

In [ ]:
main_colab_demo()

In [ ]:
filename = demo_file_upload()

# Use the global analyzer instance initialized below
demo_analysis(analyzer, filename,job_desc="healthcare")



In [ ]:
!python deploy.py

🚀 Starting deployment script...
✅ Deployment structure created successfully!


In [ ]:
import pickle
with open('.pkl', 'wb') as f:
    pickle.dump(model, f)



In [37]:
# If using Sentence Transformers
# from sentence_transformers import SentenceTransformer

# Save your model state dictionary and scaler in the training pipeline (already done in cell gfP4q_ODVsyw)
# model.save('exported_model')  # This is not needed for saving your custom scorer model

print("Model state dictionary and scaler are already saved in the training pipeline.")
print(f"Model state dict saved to: {os.path.join(config.MODEL_DIR, 'colab_model.pth')}")
print(f"Scaler saved to: {os.path.join(config.MODEL_DIR, 'scaler.pkl')}")

Model state dictionary and scaler are already saved in the training pipeline.
Model state dict saved to: /content/drive/My Drive/Resume_Analyzer/models/colab_model.pth
Scaler saved to: /content/drive/My Drive/Resume_Analyzer/models/scaler.pkl


In [41]:
import os
import torch
import pickle
from fastapi import FastAPI
from pydantic import BaseModel

# Config paths
# Assuming your notebook is running in the project directory,
# paths should be relative or use the config.MODEL_DIR
MODEL_PATH = os.path.join('models', 'best_model.pth') # Changed to load best_model.pth
SCALER_PATH = os.path.join('models', 'scaler.pkl')

# Check if paths exist (important for deployment)
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model file not found at {MODEL_PATH}")
if not os.path.exists(SCALER_PATH):
    raise FileNotFoundError(f"Scaler file not found at {SCALER_PATH}")


# Load your trained PyTorch model
# Use the correct model class definition
# class ResumeAnalyzerModel(torch.nn.Module): # Removed
#     def __init__(self, input_dim, output_dim): # Removed
#         super().__init__() # Removed
#         self.fc1 = torch.nn.Linear(input_dim, 128) # Removed
#         self.relu = torch.nn.ReLU() # Removed
#         self.fc2 = torch.nn.Linear(128, output_dim) # Removed

#     def forward(self, x): # Removed
#         x = self.relu(self.fc1(x)) # Removed
#         x = self.fc2(x) # Removed


# Initialize the model (must match training architecture)
# input_dim = 300  # Example, match your actual feature size # Removed
# output_dim = 1   # Example, adjust based on your scorer # Removed

# Initialize the actual trained model class
model = ColabResumeScorer() # Use the correct class

# Load the state dictionary
# Ensure map_location is set correctly (e.g., 'cpu' for deployment without GPU)
model.load_state_dict(torch.load(MODEL_PATH, map_location='cpu'))
model.eval() # Set model to evaluation mode

# Load the scaler
with open(SCALER_PATH, 'rb') as f:
    scaler = pickle.load(f)


print("✅ Model and scaler loaded successfully for deployment.")
print(f"Model path: {MODEL_PATH}")
print(f"Scaler path: {SCALER_PATH}")

✅ Model and scaler loaded successfully for deployment.
Model path: models/best_model.pth
Scaler path: models/scaler.pkl


In [44]:
!pip install pyngrok

Now, run the cell below to start the FastAPI server.

In [ ]:
!ngrok authtoken 333J26buLEDBpdXAi7ipeSEqe8c_4jk3RBjYQagdJv6XQW8Bk


In [ ]:
# deploy.py
import os
import sys
import uvicorn
import nest_asyncio
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from pydantic import BaseModel
from typing import Optional, Dict, List, Any

# Add project directory to sys.path to import modules
# Use the PROJECT_DIR variable defined in the notebook's first cell
# Assuming this script is intended to be run in the context where PROJECT_DIR is available
# If not, you might need to define PROJECT_DIR here or pass it as an environment variable
try:
    # Try accessing PROJECT_DIR if running in notebook context
    from __main__ import PROJECT_DIR
except ImportError:
    # Fallback if running standalone (e.g., actual deployment)
    # You might set this via env var or a config file in a real deployment scenario
    PROJECT_DIR = os.path.dirname(os.path.abspath(__file__))
    # Ensure this fallback only happens in standalone mode where __file__ is defined

sys.path.insert(0, PROJECT_DIR)


import torch
import pickle
import re

# Import your defined classes and config
# Ensure these imports match your notebook cells
# Import from __main__ if running in notebook
# Or import directly if running as a standalone script (requires these files to be present)
try:
    from __main__ import ColabDocumentParser, ColabFeatureExtractor, ColabResumeScorer, ColabConfig, config, StandardScaler
except ImportError:
    # Fallback imports for standalone script mode (if you save notebook classes as .py files)
    # from your_module import ColabDocumentParser, ColabFeatureExtractor, ColabResumeScorer, ColabConfig, config
    # from sklearn.preprocessing import StandardScaler # Assuming StandardScaler is imported here
    raise ImportError("Could not import necessary classes. Ensure you are running this in the notebook context or have saved notebook classes to .py files in PROJECT_DIR.")


# Apply nest_asyncio for Colab environments if needed (usually handled in main notebook)
# nest_asyncio.apply()

# --- Load Model and Scaler ---
# Config paths - use the paths defined in the notebook config
MODEL_PATH = os.path.join(config.MODEL_DIR, 'best_model.pth')
SCALER_PATH = os.path.join(config.MODEL_DIR, 'scaler.pkl')

# Check if paths exist
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model file not found at {MODEL_PATH}")
if not os.path.exists(SCALER_PATH):
    raise FileNotFoundError(f"Scaler file not found at {SCALER_PATH}")

# Initialize the model
# Use the same device as configured
device = config.DEVICE
model = ColabResumeScorer().to(device)

# Load the state dictionary
# Use map_location to load onto the correct device
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval() # Set model to evaluation mode

# Load the scaler
with open(SCALER_PATH, 'rb') as f:
    scaler = pickle.load(f)

print("✅ Model and scaler loaded successfully for FastAPI.")

# --- Initialize Components ---
parser = ColabDocumentParser()
extractor = ColabFeatureExtractor()

# --- FastAPI Application ---
app = FastAPI(
    title="AI Resume Analyzer API",
    description="Analyzes and scores resumes using a trained deep learning model.",
    version="1.0.0",
)

# --- API Request and Response Models ---

class AnalysisResult(BaseModel):
    success: bool
    filename: str
    scores: Dict[str, float]
    analysis: Dict[str, Any]
    recommendations: List[Dict[str, Any]]
    document_info: Dict[str, Any]
    error: Optional[str] = None

# Helper function to get ATS grade (copied from notebook)
def get_ats_grade(overall_score):
    if overall_score >= 90:
        return "A - Excellent"
    elif overall_score >= 80:
        return "B - Very Good"
    elif overall_score >= 70:
        return "C - Good"
    elif overall_score >= 60:
        return "D - Fair"
    else:
        return "F - Needs Improvement"

# Helper function to prepare model inputs (copied/adapted from notebook)
def prepare_model_inputs_for_api(features: Dict[str, Any], scaler: StandardScaler, device: torch.device):
    """Prepare features for model input in the API context"""
    # Extract embeddings
    embeddings = {}
    for emb_type in ['document', 'experience', 'education', 'skills']:
        key = f'{emb_type}_embedding'
        emb = features.get(key, torch.zeros(config.EMBEDDING_DIM)) # Use torch zeros directly
        # Ensure it's a tensor and unsqueeze for batch dim
        if not isinstance(emb, torch.Tensor):
             emb = torch.tensor(emb, dtype=torch.float32)

        embeddings[emb_type] = emb.unsqueeze(0).to(device)


    # Extract numeric features
    numeric_keys = [
        'word_count', 'section_count', 'experience_years', 'education_level',
        'total_skills', 'skill_diversity', 'contact_completeness', 'section_completeness',
        'ats_friendliness', 'readability_score', 'keyword_density',
        'has_email', 'has_phone', 'has_linkedin'
    ]

    numeric_values = []
    for key in numeric_keys:
        value = features.get(key, 0.0)
        # Apply same caps as in training
        if key == 'word_count':
            value = min(value, 2000)
        elif key == 'experience_years':
            value = min(value, 20)
        elif key == 'total_skills':
            value = min(value, 30)
        numeric_values.append(float(value))

    numeric_tensor = torch.tensor(numeric_values, dtype=torch.float32).unsqueeze(0)

    # Scale if scaler available
    if scaler:
        numeric_np = numeric_tensor.numpy()
        numeric_scaled = scaler.transform(numeric_np)
        numeric_tensor = torch.tensor(numeric_scaled, dtype=torch.float32)

    numeric_tensor = numeric_tensor.to(device)

    return {'embeddings': embeddings, 'numeric': numeric_tensor}


# Helper function to generate analysis (copied/adapted from notebook)
def generate_analysis_for_api(features, scores, document):
    """Generate comprehensive analysis for API response"""
    analysis = {
        'strengths': [],
        'weaknesses': [],
        'key_metrics': {}
    }

    # Identify strengths and weaknesses
    for component, score in scores.items():
        if component == 'overall':
            continue

        if score >= 80:
            analysis['strengths'].append(f"Excellent {component} section (Score: {score:.1f})")
        elif score >= 65:
            analysis['strengths'].append(f"Good {component} presentation (Score: {score:.1f})")
        elif score < 50:
            analysis['weaknesses'].append(f"{component.title()} section needs improvement (Score: {score:.1f})")
        elif score < 65:
            analysis['weaknesses'].append(f"{component.title()} could be enhanced (Score: {score:.1f})")

    # Key metrics
    analysis['key_metrics'] = {
        'word_count': features.get('word_count', 0),
        'experience_years': features.get('experience_years', 0),
        'total_skills': features.get('total_skills', 0),
        'ats_friendliness': f"{features.get('ats_friendliness', 0) * 100:.1f}%",
        'sections_found': len(features.get('sections_found', [])),
        'contact_complete': features.get('contact_completeness', 0) > 0.5
    }

    # Add ATS Grade
    overall_score = scores.get('overall', 0)
    analysis['ats_grade'] = get_ats_grade(overall_score)


    return analysis

# Helper function to generate recommendations (copied/adapted from notebook)
def generate_recommendations_for_api(features, scores, job_description):
    """Generate actionable recommendations for API response"""
    recommendations = []

    # Skills recommendations
    if scores.get('skills', 0) < 70:
        recommendations.append({
            'category': 'Skills',
            'priority': 'HIGH',
            'action': 'Enhance technical skills section',
            'details': [
                'Add more relevant programming languages',
                'Include industry-specific tools',
                'Group skills by category',
                'Add proficiency levels'
            ]
        })

    # Experience recommendations
    if scores.get('experience', 0) < 70:
        recommendations.append({
            'category': 'Experience',
            'priority': 'HIGH',
            'action': 'Improve experience descriptions',
            'details': [
                'Use action verbs to start bullet points',
                'Quantify achievements with numbers',
                'Show career progression',
                'Include relevant keywords'
            ]
        })

    # Education recommendations
    if scores.get('education', 0) < 60:
        recommendations.append({
            'category': 'Education',
            'priority': 'MEDIUM',
            'action': 'Strengthen education section',
            'details': [
                'Include relevant coursework',
                'Add academic achievements',
                'Mention certifications',
                'Include graduation honors'
            ]
        })

    # Formatting recommendations
    if scores.get('formatting', 0) < 65:
        recommendations.append({
                'category': 'Formatting',
            'priority': 'MEDIUM',
            'action': 'Improve ATS compatibility',
            'details': [
                'Use standard section headers',
                'Avoid complex formatting',
                'Ensure text is selectable',
                    'Use consistent formatting'
            ]
        })

    # Job matching
    if job_description and features.get('keyword_density', 0) < 0.2:
        recommendations.append({
            'category': 'Job Matching',
            'priority': 'HIGH',
            'action': 'Better align with job requirements',
            'details': [
                'Include more job-specific keywords',
                'Highlight relevant experience',
                'Customize skills for this role',
                'Match required qualifications'
            ]
        })

    return recommendations


# --- API Endpoint ---

@app.post("/analyze_resume/", response_model=AnalysisResult)
async def analyze_resume(file: UploadFile = File(...), job_description: Optional[str] = Form(None)):
    """
    Analyzes an uploaded resume file and provides scores, analysis, and recommendations.
    """
    try:
        file_content = await file.read()
        filename = file.filename

        # Step 1: Parse document
        document = parser.parse_document(file_content, filename)

        if not document.get('success', False):
            return AnalysisResult(
                success=False,
                filename=filename,
                scores={},
                analysis={},
                recommendations=[],
                document_info={},
                error=document.get('error', 'Unknown parsing error')
            )

        # Step 2: Extract features
        features = extractor.extract_all_features(document, job_description or "")

        # Step 3: Prepare model inputs
        model_inputs = prepare_model_inputs_for_api(features, scaler, device)

        # Step 4: Get predictions
        with torch.no_grad():
            predictions, _ = model(
                model_inputs['embeddings'],
                model_inputs['numeric']
            )

        # Step 5: Convert to scores (0-100)
        scores = {}
        for key, value in predictions.items():
            scores[key] = float(value.cpu().item() * 100)

        # Step 6: Generate analysis
        analysis = generate_analysis_for_api(features, scores, document)

        # Step 7: Generate recommendations
        recommendations = generate_recommendations_for_api(features, scores, job_description or "")


        return AnalysisResult(
            success=True,
            filename=filename,
            scores=scores,
            analysis=analysis,
            recommendations=recommendations,
            document_info={
                'word_count': document.get('word_count', 0),
                'sections_found': document.get('sections', []),
                'contact_info': document.get('contact', {})
            }
        )

    except Exception as e:
        import traceback
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=f"Internal Server Error: {e}")

# --- Run the API (for Colab) ---
# This part is typically run directly in the deploy.py script execution cell
# In a standard deployment, you might use a command like `uvicorn deploy:app --reload`
if __name__ == "__main__":
    # Check if running in Colab (optional, can be removed for standard deployment)
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        # Use ngrok for public URL in Colab
        from pyngrok import ngrok
        import asyncio

        # Terminate any existing ngrok tunnels
        print("Terminating existing ngrok tunnels...")
        ngrok.kill()

        # --- Add this line with your ngrok authtoken ---
        # You can get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
        # For security, it's recommended to store your token in Colab Secrets Manager (key icon)
        # and access it like: ngrok.set_auth_token(userdata.get('NGROK_AUTH_TOKEN'))
        # from google.colab import userdata # Uncomment if using Secrets Manager
        ngrok_auth_token = "333J26buLEDBpdXAi7ipeSEqe8c_4jk3RBjYQagdJv6XQW8Bk" # <<< REPLACE WITH YOUR ACTUAL AUTH TOKEN
        # ngrok_auth_token = userdata.get('NGROK_AUTH_TOKEN') # <<< Use this if using Secrets Manager
        if ngrok_auth_token == "333J26buLEDBpdXAi7ipeSEqe8c_4jk3RBjYQagdJv6XQW8Bk":
            print("\n⚠️ IMPORTANT: Please replace 'YOUR_AUTH_TOKEN' with your actual ngrok authtoken!")
            print("Sign up: https://dashboard.ngrok.com/signup")
            print("Get token: https://dashboard.ngrok.com/get-started/your-authtoken\n")
        else:
             ngrok.set_auth_token(ngrok_auth_token)

        # Start ngrok tunnel
        print("Starting ngrok tunnel...")
        public_url = ngrok.connect(8000).public_url
        print(f"✨ FastAPI app running on: {public_url}/docs")

        # Run uvicorn server
        # Use asyncio.run if not already in an event loop (Colab often is)
        # Or use nest_asyncio if required
        try:
             uvicorn.run(app, host="0.0.0.0", port=8000)
        except RuntimeError as e:
             if "Event loop is running" in str(e):
                  print("Event loop is already running, applying nest_asyncio and restarting uvicorn...")
                  nest_asyncio.apply()
                  uvicorn.run(app, host="0.0.0.0", port=8000)
             else:
                  raise # Re-raise other RuntimeErrors
    else:
        # Standard local execution
        print("Running locally...")
        uvicorn.run(app, host="0.0.0.0", port=8000)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-14' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run
    s

✅ Model and scaler loaded successfully for FastAPI.
Terminating existing ngrok tunnels...

⚠️ IMPORTANT: Please replace 'YOUR_AUTH_TOKEN' with your actual ngrok authtoken!
Sign up: https://dashboard.ngrok.com/signup
Get token: https://dashboard.ngrok.com/get-started/your-authtoken

Starting ngrok tunnel...
✨ FastAPI app running on: https://3652686c8c48.ngrok-free.app/docs


INFO:     Started server process [17940]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
